In [1]:
# %load defaults.ipy
# %load defaults.ipy
import sys
import os
for p in sys.path:
    if 'netcdf' in p.lower():
        sys.path.remove(p)
    if 'matplotlib' in p.lower():
        sys.path.remove(p)
from numpy import *
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 300
%matplotlib inline
import matplotlib.pyplot as plt
import netCDF4
from IPython.core.display import display, HTML
import matplotlib2tikz
import PIL

def console_print(message):
    message = str(message)
    with open('/dev/stdout', 'w') as f:
        f.write(message + '\n')
        f.flush()



def writeMetadata(filename, data):
    im = PIL.Image.open(filename)
    
    meta = PIL.PngImagePlugin.PngInfo()

    for key in data.keys():
        meta.add_text(key, data[key])
    im.save(filename, "png", pnginfo=meta)

    
def save(name):
    name = 'res512_' + name
    name = ''.join(ch for ch in name if ch.isalnum() or ch =='_')
    fig = plt.gcf()
    ax = plt.gca()
    ax.text(0.95, 0.01, 'By Kjetil Olsen Lye\nkjetil.o.lye@gmail.com',
         fontsize=3, color='gray',
         ha='right', va='bottom', alpha=0.5, transform=ax.transAxes)

    matplotlib2tikz.save('img_tikz/' + name + '.tikz',
           figureheight = '\\figureheight',
           figurewidth = '\\figurewidth')
    
    savenamepng = 'img/' + name + '.png'
    plt.savefig(savenamepng, bbox_inches="tight")
    
    writeMetadata(savenamepng, {'Copyright' : 'Copyright, Kjetil Olsen Lye kjetil.o.lye@gmail.com'
                               })
    
def showAndSave(name):
    save(name)
    plt.show()
    
 
    
def legendLeft():
    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [2]:
import sys
sys.path.append("/cluster/home/klye/alsvinn/build/python/")
sys.path.append("/home/kjetil/projects/alsvinn/build/python/")
import alsvinn

In [1]:
import re
def computeConvergence(basename, normP, allFiles, showSingle=False, showRes=False, statistics='mean', upTo=-1):
    # This run always fails, so we skip it
    if 'nieder' in basename or 'latin_center' in basename:
        return
    if 'kelvinhelmholtz' in basename:
        name = re.search(r'kelvinhelmholtz_([^/]+)\/', basename).group(1)
    else:
        name = re.search(r'euler_brownian_([^/]+)\/', basename).group(1)
    print(basename)
    name = name.replace("_", " ").title()
    
    N = 512
    if showSingle:
        display(HTML("<h1>%s</h1>" % name))
    
    exponents = list(range(6,12))
    Ms = 2**array(exponents)
    
    plotted_names = {}
    
    Ts = range(10)
    NT = len(Ts)
    for t in Ts:
        plt.figure(t)
        plotted_names[t] = 'cauchy_convergence_%d_%d_%s' % (normP, t, statistics)
        errors = []
        
        for M in Ms:
            filename = basename.format(M=M)
            try:
                r = alsvinn.Alsvinn(xml_file=filename, data_path=os.path.dirname(os.path.dirname(filename)))
            except Exception as e:
                print("%s failed, see error message below: " % name)
                print (e)
                return
            mean = r.get_data('rho', timestep=t, statistics=statistics)
            
            
            if M > Ms[0]:
                errors.append((sum(abs(mean-meanPrev)**normP)/mean.shape[0]**2)**(1.0/normP))
            meanPrev = mean
            if showRes:
                x,y = mgrid[0:1:N*1j, 0:1:N*1j]
                plt.pcolormesh(x,y, mean)
                plt.ylabel('$y$')
                plt.xlabel('$x$')
                plt.title('%s, $M=%d$' % (name, M))
                plt.colorbar()
                showAndSave('single_%s_%d_%s_%d' % (name, M, statistics, t))
        p = plt.loglog(Ms[:-1], errors, '-o',label=name)
        poly = polyfit(log(Ms[:-1]), log(errors), 1)
        
        plt.loglog(Ms[:-1], exp(poly[1])*Ms[:-1]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
        plt.xlabel("M")
        plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
        
        if not showSingle:
            plt.title("Cauchy convergence at $T=%d$ (normalized time)\n%s\nresolution=$512 \\times 512$" % (t, statistics))
        legendLeft()
        if showSingle:
            
            plt.title("Cauchy Convergence for %s at $T=%d$ (normalized time)\n%s\nresolution=$512 \\times 512$" % (name,t, statistics))
            showAndSave('cauchy_convergence_%d_%s_%d_%s' % (normP, name, t, statistics))
    
    for t in Ts:
        filename = basename.format(M=Ms[-1])
        try:
            r = alsvinn.Alsvinn(xml_file=filename, data_path=os.path.dirname(os.path.dirname(filename)))
        except Exception as e:
            print("%s failed, see error message below: " % name)
            print (e)
            return
        errors = []
        plt.figure(NT+t)
        plotted_names[NT+t] = 'reference_convergence_%d_%d_%s' % (normP, t, statistics)
        
        referenceSolution = r.get_data('rho', timestep=t, statistics=statistics)
        
        for M in Ms[:upTo]:
            filename = basename.format(M=M)
            try:
                r = alsvinn.Alsvinn(xml_file=filename, data_path=os.path.dirname(os.path.dirname(filename)))
            except Exception as e:
                print("%s failed, see error message below: " % name)
                print (e)
                return
            mean = r.get_data('rho', timestep=t, statistics=statistics)
               
            
            
            errors.append((sum(abs(mean-referenceSolution)**normP)/mean.shape[0]**2)**(1.0/normP))
           
        p = plt.loglog(Ms[:upTo], errors, '-o',label=name)
        poly = polyfit(log(Ms[:upTo]), log(errors), 1)
        
        plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
        plt.xlabel("M")
        plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
        if not showSingle:
            plt.title("Self convergence against reference solution $M=%d$\n$T=%d$ (normalized time)\n%s\nresolution=$512 \\times 512$" % (Ms[-1], t, statistics))
        
        legendLeft()
        if showSingle:
            plt.title("Self convergence for %s at $T=%d$ (normalized time)\nagainst reference solution with $M=%d$ samples\n%s\nresolution=$512 \\times 512$" % (name,t,Ms[-1],statistics))
            showAndSave('self_convergence_%d_%s_%d_%s' % (normP, name, t, statistics))
            
            
    for t in Ts:
        
        try:
            mc_filename = '../configs_euler512/kelvinhelmholtz_monte_carlo/m{M}/kelvinhelmholtz/kelvinhelmholtz.xml'
            filename = mc_filename.format(M=Ms[-1])
            r = alsvinn.Alsvinn(xml_file=filename, data_path=os.path.dirname(os.path.dirname(filename)))
        except Exception as e:
            print("%s failed, see error message below: " % name)
            print (e)
            return
        errors = []
        plt.figure(2*NT+t)
       
        plotted_names[2*NT+t] = 'mc_convergence_%d_%d_%s' % (normP, t, statistics)
        
        referenceSolution = r.get_data('rho', timestep=t, statistics=statistics)
        
        for M in Ms[:upTo]:
            filename = basename.format(M=M)
            try:
                r = alsvinn.Alsvinn(xml_file=filename, data_path=os.path.dirname(os.path.dirname(filename)))
            except Exception as e:
                print("%s failed, see error message below: " % name)
                print (e)
                return
            mean = r.get_data('rho', timestep=t, statistics=statistics)
               
            
            
            errors.append((sum(abs(mean-referenceSolution)**normP)/mean.shape[0]**2)**(1.0/normP))
           
        p = plt.loglog(Ms[:upTo], errors, '-o',label=name)
        poly = polyfit(log(Ms[:upTo]), log(errors), 1)
        
        plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
        plt.xlabel("M")
        plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
        if not showSingle:
            plt.title("Convergence against reference solution\ngenerated with Monte-Carlo\nwith $M=%d$ samples\n$T=%d$ (normalized time)\n%s\nresolution=$512 \\times 512$" % (Ms[-1], t, statistics))
        
        legendLeft()
        if showSingle:
            plt.title("Convergence for %s at $T=%d$ (normalized time)\nagainst reference solution MC\n%s\nresolution=$512 \\times 512$" % (name,t, statistics))
            showAndSave('mc_convergence_%d_%s_%d_%s' % (normP, name, t, statistics))
    
    
    for t in Ts:
        
        errors_max = []
        errors_mean = []
        errors_min = []
        
       
        
        referenceSolution = r.get_data('rho', timestep=t, statistics=statistics)
        
        for M in Ms[:upTo]:
            filename = basename.format(M=M)
            try:
                r = alsvinn.Alsvinn(xml_file=filename, data_path=os.path.dirname(os.path.dirname(filename)))
            except Exception as e:
                print("%s failed, see error message below: " % name)
                print (e)
                return
            mean = r.get_data('rho', timestep=t, statistics=statistics)
               
            error_mean = 0
            error_min = 100000000
            error_max = -error_min
            number_of_files = 0
            for f in allFiles:
                try:
                    
                    filename = f.format(M=Ms[-1])
                    r = alsvinn.Alsvinn(xml_file=filename, data_path=os.path.dirname(os.path.dirname(filename)))
                    meanReference = r.get_data('rho', timestep=t, statistics=statistics)
                except Exception as e:
                    print("%s failed, see error message below: " % f)
                    print(e)
                    continue
                error = (sum(abs(mean-meanReference)**normP)/mean.shape[0]**2)**(1.0/normP)
                
                error_mean += error
                error_min = min(error_min, error)
                error_max = max(error_max, error)
                number_of_files +=1
            error_mean /= number_of_files
            errors_max.append(error_max)
            errors_min.append(error_min)
            errors_mean.append(error_mean)
                
        if not showSingle:
            plt.figure(6*NT+t)
            plotted_names[6*NT+t] = 'worst_convergence_%d_%d_%s' % (normP, t, statistics)
            p = plt.loglog(Ms[:upTo], errors_max, '-o',label=name)
            poly = polyfit(log(Ms[:upTo]), log(errors_max), 1)
            
            plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
            plt.xlabel("M")
            plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
            legendLeft()
            plt.title("\"Worst case\" convergence against reference solution\n\nwith $M=%d$ samples\n$T=%d$ (normalized time)\n%s\nresolution=$512 \\times 512$" % (Ms[-1], t,statistics))
        
        
            plt.figure(8*NT+t)
            plotted_names[8*NT+t] = 'best_convergence_%d_%d_%s' % (normP, t, statistics)
            p = plt.loglog(Ms[:upTo], errors_min, '-o',label=name)
            poly = polyfit(log(Ms[:upTo]), log(errors_min), 1)
            
            plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
            plt.xlabel("M")
            plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
            legendLeft()
            plt.title("\"Best case\" convergence against reference solution\n\nwith $M=%d$ samples\n$T=%d$ (normalized time)\n%s\nresolution=$512 \\times 512$" % (Ms[-1], t,statistics))
        
        
            plt.figure(10*NT+t)
            plotted_names[10*NT+t] = 'mean_convergence_%d_%d_%s' % (normP, t, statistics)
            p = plt.loglog(Ms[:upTo], errors_mean, '-o',label=name)
            poly = polyfit(log(Ms[:upTo]), log(errors_mean), 1)
            
            plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
            plt.xlabel("M")
            plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
            legendLeft()
            plt.title("\"Average\" convergence against reference solution\n\nwith $M=%d$ samples\n$T=%d$ (normalized time)\n%s\nresolution=$512 \\times 512$" % (Ms[-1], t, statistics))
        
        else:
            p = plt.loglog(Ms[:upTo], errors_max, '-o',label="worst case error")
            poly = polyfit(log(Ms[:upTo]), log(errors_max), 1)
            
            plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
            plt.xlabel("M")
            plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
            legendLeft()
            
            
            p = plt.loglog(Ms[:upTo], errors_min, '-o',label="best case error")
            poly = polyfit(log(Ms[:upTo]), log(errors_min), 1)
            
            plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
            plt.xlabel("M")
            plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
            legendLeft()
            
        
            
            p = plt.loglog(Ms[:upTo], errors_mean, '-o',label="average error")
            poly = polyfit(log(Ms[:upTo]), log(errors_mean), 1)
            
            plt.loglog(Ms[:upTo], exp(poly[1])*Ms[:upTo]**poly[0], '--', color=p[0].get_color(), label='$O(M^{%.3f})$' % poly[0])
            plt.xlabel("M")
            plt.ylabel("Error ($||\\cdot||_{L^{%d}}$)" % normP)
            legendLeft()
        
            plt.title("Convergence for %s at $T=%d$ (normalized time)\nagainst reference solution MC\n%s\nresolution=$512 \\times 512$" % (name,t, statistics))
            showAndSave('best_worst_mean_convergence_%d_%s_%d_%s' % (normP, name, t, statistics))
    return plotted_names

# Brownian initial data

Here we let the initial data be given as (truncated) Brownian motion, and we set
$$u^x_0(\omega, x,y) = B^1_{x,y}(\omega)\qquad u^y_0(\omega, x,y)=B^2_{x,y}(\omega)$$
for two independent two dimensional Brownian motions $B^1$ and $B^2$. We set the initial pressure and density to be constant.

In the approximation, we use $512^2$ dimensions. Not all QMC rules are able to handle this dimension, they are skipped.

In [ ]:
import glob

for H in [0.1, 0.5, 0.75]:
    display(HTML('<h1>H=%f</h1>' % H))
    allFiles = []
    console_print("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H)
    console_print(glob.glob("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H))
    for f in glob.glob("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H):
        if not 'nieder' in f and not 'latin_center' in f:
            allFiles.append(f + "/m{M}/euler_brownian/euler_brownian.xml")
    
    
    console_print("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H)
    for basename in glob.glob("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H):
        names = computeConvergence(basename + "/m{M}/euler_brownian/euler_brownian.xml", 2, allFiles)
        
    for (figure_number, filename) in names.items():
        plt.figure(figure_number)
        legendLeft()
        save(filename)
    plt.show()
    

    
    

# Variance


In [ ]:
import glob

for H in [0.1, 0.5, 0.75]:
    display(HTML('<h1>H=%f</h1>' % H))
    allFiles = []
    for f in glob.glob("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H):
        if not 'nieder' in f and not 'latin_center' in f:
            allFiles.append(f + "/m{M}/euler_brownian/euler_brownian.xml")
    
    
    
    for basename in glob.glob("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H):
        names = computeConvergence(basename + "/m{M}/euler_brownian/euler_brownian.xml", 2, allFiles, statistics='variance')
        
    for (figure_number, filename) in names.items():
        plt.figure(figure_number)
        legendLeft()
        save(filename)
    plt.show()
    

# Mean examples Brownian


In [ ]:
for H in [0.1, 0.5, 0.75]:
    display(HTML('<h1>H=%f</h1>' % H))
    allFiles = []
    for f in glob.glob("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H):
        if not 'nieder' in f and not 'latin_center' in f:
            allFiles.append(f + "/m{M}/euler_brownian/euler_brownian.xml")
    
    
    
    for basename in glob.glob("../configs_euler_brownian/N64/H_%f/euler_brownian*" % H):
        names = computeConvergence(basename + "/m{M}/euler_brownian/euler_brownian.xml", 2, allFiles, True, True)
  
    plt.show()
    

# Brownian initial data

Here we let the initial data be given as (truncated) Brownian motion, and we set
$$u^x_0(\omega, x,y) = B^1_{x,y}(\omega)\qquad u^y_0(\omega, x,y)=B^2_{x,y}(\omega)$$
for two independent two dimensional Brownian motions $B^1$ and $B^2$. We set the initial pressure and density to be constant.

In the approximation, we use $512^2$ dimensions. Not all QMC rules are able to handle this dimension, they are skipped.

In [ ]:
import glob

#for basename in glob.glob("../configs/brownian_*"):
#    computeConvergence(basename + "/brownian.xml")

In [4]:
import glob

#for basename in glob.glob("../configs/brownian_*"):
#    computeConvergence(basename + "/brownian.xml", True)
from numpy import *
a=array([1,2,3,4])
a[:None]

array([1, 2, 3, 4])